In [1]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

In [2]:
meetings_df = pd.read_csv(os.path.join(LOCAL_PATH, "intermediate_data/cpc/meetings-manifest.csv"))
DATES = sorted(list(meetings_df['date']))
meta_df = meetings_df.copy()

In [3]:
meta_df['total_pages'] = meta_df['agenda_pages'] + meta_df['minutes_pages'] + meta_df['supdocs_pages']

In [4]:
meta_df['total_supplemental_docs'] = 0
for idx, row in meta_df.iterrows():
    year = row['year']
    date = row['date']
    filepath = os.path.join(LOCAL_PATH, f"intermediate_data/cpc/{year}/{date}/supplemental-docs.pkl")
    if os.path.exists(filepath):
        supplemental_docs_df = pd.read_pickle(filepath)
        meta_df.loc[idx, 'total_supplemental_docs'] = len(supplemental_docs_df)

In [5]:
meta_df['total_agenda_items'] = 0
meta_df['total_agenda_cases'] = 0
for idx, row in meta_df.iterrows():
    year = row['year']
    date = row['date']
    filepath = os.path.join(LOCAL_PATH, f"intermediate_data/cpc/{year}/{date}/agenda-items.pkl")
    if os.path.exists(filepath):
        agenda_items_df = pd.read_pickle(filepath)
        meta_df.loc[idx, 'total_agenda_items'] = len(agenda_items_df)
        meta_df.loc[idx, 'total_agenda_cases'] = len(agenda_items_df.loc[agenda_items_df['is_casenum']])

In [6]:
n_meetings = len(meta_df)
n_years = len(meta_df['year'].unique())
min_year = meta_df['year'].astype('int').min()
max_year = meta_df['year'].astype('int').max()
n_pages = meta_df['total_pages'].sum()
n_supplemental_docs = meta_df['total_supplemental_docs'].sum()
n_agenda_items = meta_df['total_agenda_items'].sum()
n_agenda_cases = meta_df['total_agenda_cases'].sum()

print(f"{n_meetings} meetings")
print(f"across {n_years} years from {min_year} to {max_year}")
print(f"totaling {n_agenda_items:,g} agenda items")
print(f"and {n_agenda_cases:,g} planning department cases on the agenda")
print(f"and {n_supplemental_docs:,g} supplemental documents")
print(f"total pages of documents: {n_pages:,g}")


156 meetings
across 7 years from 2018 to 2024
totaling 1,506 agenda items
and 732 planning department cases on the agenda
and 6,482 supplemental documents
total pages of documents: 23,712
